In [2]:
!pip install streamlit

  Using cached streamlit-1.40.2-py2.py3-none-any.whl.metadata (8.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [4]:
!pip install scikit-learn

In [5]:
!pip install transformers

In [8]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [14]:
%%writefile app.py
import streamlit as st
from datasets import load_dataset
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load MedQuAD dataset from Hugging Face
@st.cache_data
def load_medquad_data():
    dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset", split="train")
    questions = dataset["Question"]
    answers = dataset["Answer"]
    return questions, answers

questions, answers = load_medquad_data()

# TF-IDF Vectorization for questions
@st.cache_resource
def initialize_vectorizer():
    vectorizer = TfidfVectorizer().fit(questions)
    question_vectors = vectorizer.transform(questions)
    return vectorizer, question_vectors

vectorizer, question_vectors = initialize_vectorizer()

# Retrieval function
def retrieve_answer(user_question):
    user_vector = vectorizer.transform([user_question])
    similarities = cosine_similarity(user_vector, question_vectors).flatten()
    best_match_idx = np.argmax(similarities)
    return answers[best_match_idx], questions[best_match_idx]

# Load Hugging Face's NER pipeline with a model trained for medical entity recognition
@st.cache_resource
def load_ner_pipeline():
    # Use a medical NER model from Hugging Face (like `dmis-lab/biobert-v1.1` or `cogstack/biomedical-ner`)
    ner_pipeline = pipeline("ner", model="dmis-lab/biobert-v1.1", tokenizer="dmis-lab/biobert-v1.1")
    return ner_pipeline

ner_pipeline = load_ner_pipeline()

# Medical Entity Recognition using Hugging Face
def recognize_entities(text):
    entities = ner_pipeline(text)
    medical_entities = [{"text": entity["word"], "label": entity["entity"]} for entity in entities]
    return medical_entities

# Streamlit Interface
st.title("Medical Q&A Chatbot")
st.sidebar.title("About")
st.sidebar.info(
    "This chatbot uses the MedQuAD dataset from Hugging Face to provide answers to medical questions. "
    "It also recognizes medical entities in the input text using a transformer-based NER model."
)

# User input
user_question = st.text_input("Ask a medical question:")

if user_question:
    # Retrieve the best answer
    answer, matched_question = retrieve_answer(user_question)

    # Display results
    st.subheader("Answer:")
    st.write(answer)

    st.subheader("Matched Question:")
    st.write(matched_question)

    # Recognize entities
    st.subheader("Recognized Medical Entities:")
    entities = recognize_entities(user_question)
    if entities:
        for entity in entities:
            st.write(f"{entity['text']} - {entity['label']}")
    else:
        st.write("No entities recognized.")


Writing app.py


In [11]:
!pip install pyngrok

  Using cached pyngrok-7.2.1-py3-none-any.whl.metadata (8.3 kB)


In [13]:
!ngrok config add-authtoken 2pQvixZCRRnA2pe5Ow9Xomj1GCl_7rgJxvHhn7jqt1kyaQYm6

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [16]:
from pyngrok import ngrok
import os
os.system('streamlit run app.py &')
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

Streamlit app is live at: NgrokTunnel: "https://8dd2-35-192-107-5.ngrok-free.app" -> "http://localhost:8501"
